The micro-scale structure describes the positions and roles played by specific nodes within a greater network. Such analysis can be used to identify influential individuals, bottlenecks in flows, and convenient locations for assembling information or resources.
* **Centrality-** Quantifying a node's structural properties using centrality measures
* **Betweenness centrality-** Identifying nodes that act like bridges using the betweenness centrality
* **Eigenvector centrality-** Quantifying the distance between a node and the rest of the network using closeness centrality
* **Local clustering-** Quantifying the interconnectedness of a node's neighborhood


## Centrality - Finding Key Nodes
Centrality is often introduced as a measure of importance, but there are many ways in which a node can be important. One of the simplest centrality measures is **degree centrality**. A node degree of centrality is just the number of neighbors it has (in a directed network, there are both in-degree and out-degree). 
In a social network, degree centrality is a measure of popularity, and might be a good way to guess who throws the best parties etc

## Bridges, Brokers, and Bottlenecks - Betweenness centrality
In a complex social network, such as organizations and social movements, the individuals who connect different parts of the network have the greatest ability to filter, amplify, and alter information. Such individuals are called **brokers**, while edges that span distant parts of the network are called **bridges**. This information isn't limited to social networks. In flow networks, such as railroads, water pipes, and telecommunications systems - nodes connecting distant parts of a network can act as a bottleneck, limiting the amount of flow. Identifying such bottlenecks makes it possible to increase their capacity and protect them from failures and attacks. 

To understand betweenness centrality, it is necessary to understand the concept of shortest path. A path is a sequence of nodes, moving along edges from neighbor to neighbor. Betweenness centrality is based on the assumption that the greater the number of shortest paths pass through a node(or edge), the more it acts as a broker(or bridge). To calculate betweenness centrality, the shortest paths between each pair of nodes are found. The betweenness centrality value for a node or edge is just the number of these paths that pass through it. There are a couple of caveats. First, by convention, paths are not considered to pass through their endpoints and do not contribute to betweenness centrality.

Betweenness centrality is based on the assumption that the greater the number of shortest paths pass through a node(or edge), the more it acts as a broker(or bridge). If there are two shortest paths of the same length, each path contributes 1/2 to the betweenness centrality of its nodes/edges(or 1/3 if there are 3 paths).

Let us cover a covert network, namely of British suffragettes in the early 20th century. These networks are particularly interesting to study because there is a tension between the need for secrecy and the need to spread information throughout the network. The following example comes from the arrest records of British suffragettes. These records form an affiliation network between individuals and mass arrrests.

In [1]:
from networkx.algorithms import bipartite
from networkx import NetworkXError
import networkx as nx
from pathlib import Path



# create an empty affiliation network and list of people
B = nx.Graph()
people = set()

data_dir = "../data/crossley2012/50_ALL_2M.csv"
with open(data_dir) as f:
    # parse header
    events = next(f).strip().split(",")[1:]
    for row in f:
        parts = row.strip().split(",")
        person = parts[0]
        people.add(person)
        for j, value in enumerate(parts[1:]):
            if value != "0":
                B.add_edge(person, events[j], weights = int(value))

# project into person-person co-affiliation network
G = bipartite.projected_graph(B, people)


In [2]:
# calculate the betweenness centrality
betweenness = nx.betweenness_centrality(
    G,
    normalized=False
)

sorted(betweenness.items(), key=lambda x:x[1], reverse=True)[0:10]

[('Maud Joachim', 52896.53324605239),
 ('Ada Wright', 26344.263264276873),
 ('Patricia Woodlock', 24774.923422322485),
 ('Emily Duval', 19517.90621411947),
 ('Mary Leigh', 19404.225833772107),
 ('Mabel Capper', 18221.362811581734),
 ('Sylvia Pankhurst', 18127.59688636898),
 ('Elsie Evans', 15674.806298703472),
 ('Winifred Mayo', 15600.989680321347),
 ('Vera Wentworth', 13233.504078942527)]

High betweenness centrality suggests that these individuals were important information brokers in the suffragette movement

## Hubs - Eigenvector Centrality
The more well-connected a node, the higher the eigenvector centrality

In [4]:
eigenvector = nx.eigenvector_centrality(G)
sorted(eigenvector.items(), key=lambda x: x[1], reverse=True)[0:10]

[('Maud Joachim', 0.11587964174472977),
 ('Caroline A Downing', 0.11437066100686202),
 ('Kitty Marion', 0.11344996012448637),
 ('Mabel Capper', 0.10991776240126291),
 ('Annie Bell', 0.10834705221110304),
 ('Grace Chappelow', 0.10818185244249963),
 ('Winifred Mayo', 0.10803831965810351),
 ('Ellen Pitfield', 0.10518714292397999),
 ('Dorothy Agnes Bowker', 0.10493919222545887),
 ('Mrs Maud Fussell', 0.10490326319130666)]

Individuals with high eigenvector centrality create many short paths between others, but not necessarily the shortest paths

## Closeness Centrality
The farness of a node is the sum of distances between that node and all other nodes. So a node with high closeness centrality is literally close to the other nodes. Nodes with high closeness have, on average, short paths to many other nodes, which can be helpful for disseminating resources quickly.

In [5]:
closeness = nx.closeness_centrality(G)
sorted(closeness.items(), key=lambda x: x[1], reverse=True)[0:10]

[('Maud Joachim', 0.5357241748956739),
 ('Caroline A Downing', 0.5009438937877011),
 ('Winifred Mayo', 0.5009438937877011),
 ('Mabel Capper', 0.5006919099377073),
 ('Kitty Marion', 0.49793672684150186),
 ('Ada Wright', 0.4898501559823633),
 ('Patricia Woodlock', 0.4886477746471095),
 ('Vera Wentworth', 0.48769011119851163),
 ('Evelyn Whurry', 0.4874512815652116),
 ('Annie Bell', 0.4869743233640714)]

## Local Clustering
Betweenness, eigenvector, and closeness centrality all characterize a node by its relation to other nodes in the network. It is often useful to consider whether a node's neighbors tend to be connected to each other. In a social network, the question translates to asking whether the friend of a friend is also your friend, a property known as **transitivity**. The results of such relationships are triangles: three nodes, all mutually connected. When strong clustering is present, it often suggests robustness, and redundancy in a network - if one edge disappears, a path still exists via the other two. Clustering is measured via **local clustering coefficient**, defined as the fraction of all pairs of a node's neighbors that have an edge between them. 


In [6]:
triangles = nx.triangles(G)
sorted(triangles.items(), key=lambda x: x[1], reverse=True)[0:10]

[('Maud Joachim', 19687),
 ('Caroline A Downing', 18201),
 ('Kitty Marion', 17696),
 ('Mabel Capper', 16811),
 ('Winifred Mayo', 16455),
 ('Annie Bell', 16065),
 ('Grace Chappelow', 16018),
 ('Ellen Pitfield', 14910),
 ('Mrs Maud Fussell', 14841),
 ('Dorothy Agnes Bowker', 14750)]

In [7]:
# use the clustering() to find the local clustering coefficient for these nodes
clustering = nx.clustering(G)
[(x, clustering[x]) for x in sorted(people, key=lambda x:eigenvector[x], reverse=True)[0:10]]

[('Maud Joachim', 0.23595330552759),
 ('Caroline A Downing', 0.34999903851700864),
 ('Kitty Marion', 0.3670988486671507),
 ('Mabel Capper', 0.33992518451117176),
 ('Annie Bell', 0.4233201581027668),
 ('Grace Chappelow', 0.43461037551551984),
 ('Winifred Mayo', 0.3480477177545582),
 ('Ellen Pitfield', 0.4828993392926545),
 ('Dorothy Agnes Bowker', 0.5058125578683859),
 ('Mrs Maud Fussell', 0.5006071645415908)]